# 誤差逆伝播法に対応したNNを実装する

In [2]:
import sys, os
import numpy as np
from collections import OrderedDict

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.py、layers.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.layers import SoftmaxWithLoss, Affine, ReLU

ローカル環境での実行です


### [演習]
* 以下の誤差逆伝播法に対応したNNクラスを完成させましょう。

In [3]:
# ヒント

# OrderedDictを用いると、順序を保持したdictを生成することができる
# ただし、Pythonバージョン3.6以降は、普通のdictでも順序が保存される
dic1 = OrderedDict()
dic1["a"] = 1
dic1["c"] = 3
dic1["b"] = 2
for key, value in dic1.items():
    print(key, value)
print()

# dictから逆順でvalueを取り出す方法
layers = list(dic1.values())
layers.reverse() # 順序を逆転させる
print(layers)

a 1
c 3
b 2

[2, 3, 1]


In [4]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict() # 順番付きdict形式. 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss() # 出力層
        
    def predict(self, x):
        """
        予測関数
        x : 入力
        """
        for layer in self.layers.values():
            # 入力されたxを上書きしていく = 順伝播計算
            # ここのforwardは common/layers.pyにおいて実装されている
            # from common.layers import SoftmaxWithLoss, Affine, ReLU
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数
        x:入力データ, t:正解データ
        """
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        """
        正解率
        """
        # 予測 返り値は正規化されていない実数
        y = self.predict(x)
        
        #正規化されていない実数をもとに、最大値になるindexに変換する
        y = np.argmax(y, axis=1)
        
        if t.ndim != 1 : 
            """
            one-hotベクトルの場合、正解データをindexに変換する
            """
            t = np.argmax(t, axis=1)
        
        # 正解率
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def gradient(self, x, t):
        """
        全パラメータの勾配を計算
        """
        
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = self.lastLayer.backward() # 出力層
        
        ## doutを逆向きに伝える 
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # dW, dbをgradsにまとめる
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [5]:
# 学習用データ
x = np.array([[1, 2, 3],
            [3, 2, 1]])
t = np.array([[0, 0, 1],
            [1, 0, 0]])

tnet = TwoLayerNet( input_size=3, hidden_size=5, output_size=3)

print("predict=\n", tnet.predict(x))
print()
print("loss=\n", tnet.loss(x, t))
print()
print("accuracy=", tnet.accuracy(x, t))
print()
print("gradient=\n",)
grads = tnet.gradient(x, t)
for key, value in grads.items():
    print(key)
    print(value)

predict=
 [[ 4.35860486e-04 -2.45113470e-04  9.16219859e-05]
 [ 4.61896148e-04 -1.70865992e-04 -8.12346205e-05]]

loss=
 1.0984172959455822

accuracy= 0.5

gradient=

W1
[[-0.01218618  0.          0.00088886 -0.01201062  0.        ]
 [-0.00251678  0.          0.00177773 -0.00944842  0.        ]
 [ 0.00715262  0.          0.00266659 -0.00688622  0.        ]]
b1
[-0.00125839  0.          0.00088886 -0.00472421  0.        ]
W2
[[-6.16258131e-03  5.31202016e-03  8.50561152e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.21661331e-06  7.21170066e-06 -1.44283140e-05]
 [-8.54584459e-04  8.81696424e-03 -7.96237978e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
b2
[-0.16654437  0.33323666 -0.16669229]
